# Titanic_Spark

In [1]:
val spk = spark
import spk.implicits._
spark.conf.set("spark.sql.shuffle.partitions", "8")

spk = org.apache.spark.sql.SparkSession@18905834


Spark local-1535716557168: Some(http://192.168.88.103:4040)

In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [3]:
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.Row
import org.apache.spark.SparkContext

In [ ]:
val schemaArray = StructType(Array(
    StructField("PassengerId", IntegerType, true),
    StructField("Survived", IntegerType, true),
    StructField("Pclass", IntegerType, true),
    StructField("Name", StringType, true),
    StructField("Sex", StringType, true),
    StructField("Age", FloatType, true),
    StructField("SibSp", IntegerType, true),
    StructField("Parch", IntegerType, true),
    StructField("Ticket", StringType, true),
    StructField("Fare", FloatType, true),
    StructField("Cabin", StringType, true),
    StructField("Embarked", StringType, true)
  ))

val path = "/Titanic/train.csv"
val df = spark.read
              .format("csv")
              .option("header", true)
              .schema(schemaArray)
              .load(path)
df.cache()

In [4]:
df.printSchema

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
df.take(5)

1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NULL,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NULL,S


## Data Transformation

### Non-Pipeline

In [22]:
//Cabin，用“U”填充null，并提取Cabin的首字母
val df2 = df.na.fill("U", Seq("Cabin"))
  .withColumn("Cabin", substring($"Cabin", 0, 1))

df2 = [PassengerId: int, Survived: int ... 10 more fields]


[PassengerId: int, Survived: int ... 10 more fields]

In [5]:
def transCabin(df: Dataset[Row]): Dataset[Row] = {
    df.na.fill("U", Seq("Cabin"))
        .withColumn("Cabin", substring($"Cabin", 0, 1))
}

transCabin: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [6]:
val df2 = transCabin(df)

df2 = [PassengerId: int, Survived: int ... 10 more fields]


[PassengerId: int, Survived: int ... 10 more fields]

In [9]:
df2.select("Cabin").show(5)

+-----+
|Cabin|
+-----+
|    U|
|    C|
|    U|
|    C|
|    U|
+-----+
only showing top 5 rows



In [23]:
//Ticket, 提取船票的号码，如“A/5 21171”中的21171
val df3 = df2.withColumn("Ticket", split($"Ticket", " "))
    .withColumn("Ticket", $"Ticket"(size($"Ticket").minus(1)))
    .filter($"Ticket" =!= "LINE")//去掉某种特殊的船票

df3 = [PassengerId: int, Survived: int ... 10 more fields]


[PassengerId: int, Survived: int ... 10 more fields]

In [9]:
df3.select("Ticket").show(5)

+-------+
| Ticket|
+-------+
|  21171|
|  17599|
|3101282|
| 113803|
| 373450|
+-------+
only showing top 5 rows



In [24]:
//Ticket, 对船票号进行分类，小于四位号码的为“1”，四位号码的以第一个数字开头，后面接上“0”，大于4位号码的，取前三个数字开头。如21171变为211
def ticketTrans(ticket: String): String = {
    if (ticket.length < 4) {
        return "1"
    } else if (ticket.length == 4){
        return ticket(0)+"0"
    } else {
        return ticket.slice(0, 3)
    }
}
val ticketTransUdf = udf(ticketTrans(_:String):String)
val df4 = df3.withColumn("Ticket", ticketTransUdf($"Ticket"))
df4.cache()

ticketTransUdf = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
df4 = [PassengerId: int, Survived: int ... 10 more fields]


ticketTrans: (ticket: String)String


[PassengerId: int, Survived: int ... 10 more fields]

In [25]:
//Ticket, 将数量小于等于5的类别统一归为“0”
val filterList = df4.groupBy($"Ticket").count()
  .filter($"count" <= 5)
  .map(row => row.getString(0))
  .collect.toList

val filterList_bc = sc.broadcast(filterList)
def ticketTransAdjust: (String => String) = {
    subticket => {
        if (filterList_bc.value.contains(subticket)) "0"
            else subticket
    }
}
val ticketTransAdjustUdf = udf(ticketTransAdjust)

val df5 = df4.withColumn("Ticket", ticketTransAdjustUdf($"Ticket"))

filterList = List(245, 324, 226, 285, 453, 372, 149, 80, 340, 143, 398, 348, 231, 352, 228, 363, 362, 127, 488, 132, 211, 90, 293, 295, 172, 104, 198, 290, 229, 218, 122, 50, 173, 368, 243, 118, 214, 319, 384, 336, 240, 187, 241, 124, 235, 346, 374, 312, 267, 376, 130, 342, 263, 546, 373, 335, 116, 331, 371, 265, 284, 142, 220, 169, 386, 323, 278, 205, 219, 334, 281, 343, 272, 365, 119, 314, 341, 394, 653, 185, 270, 223, 545, 358, 234, 236, 286, 282, 297, 233, 383)
filterList_bc = Broadcast(6)
ticketTransAdjust = > String
ticketTransAdjustUdf = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
df5 = [PassengerId: ...


[PassengerId: int, Survived: int ... 10 more fields]

In [8]:
def transTicket(sc: SparkContext, df: Dataset[Row]): Dataset[Row] = {
    
//     def ticketTrans(ticket: String): String = {
//         if (ticket.length < 4) {
//             return "1"
//         } else if (ticket.length == 4){
//             return ticket(0)+"0"
//         } else {
//             return ticket.slice(0, 3)
//         }
//     }
//     val ticketTransUdf = udf(ticketTrans(_:String):String)
    
    
    
//     def ticketTransAdjust: (String => String) = {
//         subticket => {
//             if (filterList_bc.value.contains(subticket)) "0"
//                 else subticket
//         }
//     }
//     val ticketTransAdjustUdf = udf(ticketTransAdjust)
    
    val ticketTransUdf = udf((ticket: String) => {
        if (ticket.length < 4) {
             "1"
        } else if (ticket.length == 4){
             ticket(0)+"0"
        } else {
             ticket.slice(0, 3)
        }
    })
    
    val medDF = df.withColumn("Ticket", split($"Ticket", " "))
    .withColumn("Ticket", $"Ticket"(size($"Ticket").minus(1)))
    .filter($"Ticket" =!= "LINE")
    .withColumn("Ticket", ticketTransUdf($"Ticket"))
    
    val filterList = medDF.groupBy($"Ticket").count()
  .filter($"count" <= 5)
  .map(row => row.getString(0))
  .collect.toList

    val filterList_bc = sc.broadcast(filterList)
    
    val ticketTransAdjustUdf = udf((subticket: String) => {
        if (filterList_bc.value.contains(subticket)) "0"
        else subticket
    })
    
    medDF.withColumn("Ticket", ticketTransAdjustUdf($"Ticket"))
}

transTicket: (sc: org.apache.spark.SparkContext, df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [9]:
val df3 = transTicket(sc, df2)

df3 = [PassengerId: int, Survived: int ... 10 more fields]


[PassengerId: int, Survived: int ... 10 more fields]

In [10]:
df3.columns.size

12

In [12]:
df5.groupBy("Ticket").count().orderBy($"count".desc).show(100)

+------+-----+
|Ticket|count|
+------+-----+
|     0|  195|
|    20|   97|
|   349|   53|
|   310|   48|
|   347|   47|
|   113|   47|
|   174|   22|
|   345|   19|
|   175|   19|
|   177|   18|
|   350|   18|
|    30|   15|
|   315|   14|
|   250|   14|
|   199|   14|
|   364|   14|
|   176|   13|
|   135|   13|
|    70|   12|
|   248|   12|
|   370|   12|
|    40|   12|
|   392|   11|
|   117|   11|
|   330|   11|
|   110|   10|
|   244|   10|
|    10|   10|
|    60|    9|
|     1|    9|
|   367|    9|
|   237|    8|
|   239|    8|
|   369|    8|
|   111|    8|
|   291|    8|
|   230|    8|
|   112|    8|
|   382|    7|
|   148|    6|
+------+-----+



In [9]:
sc.getClass

class org.apache.spark.SparkContext

In [26]:
//Embarked, 用“S”填充null
val df6 = df5.na.fill("S", Seq("Embarked"))

df6 = [PassengerId: int, Survived: int ... 10 more fields]


[PassengerId: int, Survived: int ... 10 more fields]

In [11]:
def transEmbarked(df: Dataset[Row]): Dataset[Row] = {
    df.na.fill("S", Seq("Embarked"))
}

transEmbarked: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [12]:
val df4 = transEmbarked(df3)

df4 = [PassengerId: int, Survived: int ... 10 more fields]


[PassengerId: int, Survived: int ... 10 more fields]

In [27]:
//Name, 对头衔进行归类
val regex = ".*, (.*?)\\..*"

val titlesMap = Map(
    "Capt"-> "Officer",
    "Col"-> "Officer",
    "Major"-> "Officer",
    "Jonkheer"-> "Royalty",
    "Don"-> "Royalty",
    "Sir" -> "Royalty",
    "Dr"-> "Officer",
    "Rev"-> "Officer",
    "the Countess"->"Royalty",
    "Mme"-> "Mrs",
    "Mlle"-> "Miss",
    "Ms"-> "Mrs",
    "Mr" -> "Mr",
    "Mrs" -> "Mrs",
    "Miss" -> "Miss",
    "Master" -> "Master",
    "Lady" -> "Royalty"
)

val titlesMap_bc = sc.broadcast(titlesMap)

val df7 = df6.withColumn("Title", regexp_extract(($"Name"), regex, 1))
  .na.replace("Title", titlesMap_bc.value)

// def titleAdjust: (String => String) = {
//     subticket => titlesMap_bc.value.getOrElse(subticket, "U")}
// val titleAdjustUdf = udf(titleAdjust)

// val df7 = df6.select(expr("*"), regexp_extract(($"Name"), regex, 1).alias("Name_clean"))
//     .withColumn("Name_final", titleAdjustUdf($"Name_clean"))

regex = .*, (.*?)\..*
titlesMap = Map(Master -> Master, Capt -> Officer, Mr -> Mr, Dr -> Officer, Don -> Royalty, Rev -> Officer, Lady -> Royalty, Mrs -> Mrs, Miss -> Miss, Mlle -> Miss, Major -> Officer, Col -> Officer, Mme -> Mrs, Sir -> Royalty, the Countess -> Royalty, Jonkheer -> Royalty, Ms -> Mrs)
titlesMap_bc = Broadcast(7)
df7 = [PassengerId: int, Survived: int ... 11 more fields]


[PassengerId: int, Survived: int ... 11 more fields]

In [13]:
def extractTitle(sc: SparkContext, df: Dataset[Row]): Dataset[Row] = {
  val regex = ".*, (.*?)\\..*"

  val titlesMap = Map(
      "Capt"-> "Officer",
      "Col"-> "Officer",
      "Major"-> "Officer",
      "Jonkheer"-> "Royalty",
      "Don"-> "Royalty",
      "Sir" -> "Royalty",
      "Dr"-> "Officer",
      "Rev"-> "Officer",
      "the Countess"->"Royalty",
      "Mme"-> "Mrs",
      "Mlle"-> "Miss",
      "Ms"-> "Mrs",
      "Mr" -> "Mr",
      "Mrs" -> "Mrs",
      "Miss" -> "Miss",
      "Master" -> "Master",
      "Lady" -> "Royalty"
  )

  val titlesMap_bc = sc.broadcast(titlesMap)

  df.withColumn("Title", regexp_extract(($"Name"), regex, 1))
    .na.replace("Title", titlesMap_bc.value)
}

extractTitle: (sc: org.apache.spark.SparkContext, df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [14]:
val df5 = extractTitle(sc, df4)

df5 = [PassengerId: int, Survived: int ... 11 more fields]


[PassengerId: int, Survived: int ... 11 more fields]

In [16]:
df5.columns

[PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Title]

In [22]:
df5.select("Title", "Name").show(5, false)

+-----+---------------------------------------------------+
|Title|Name                                               |
+-----+---------------------------------------------------+
|Mr   |Braund, Mr. Owen Harris                            |
|Mrs  |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|
|Miss |Heikkinen, Miss. Laina                             |
|Mrs  |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |
|Mr   |Allen, Mr. William Henry                           |
+-----+---------------------------------------------------+
only showing top 5 rows



In [16]:
df7.groupBy("Pclass", "Title").mean("Age").select(concat($"Title", $"Pclass"), $"avg(Age)").show

+---------------------+------------------+
|concat(Title, Pclass)|          avg(Age)|
+---------------------+------------------+
|                Miss2|         22.390625|
|                  Mr3|28.662222222222223|
|                 Mrs3|33.515151515151516|
|                Miss3|  16.1231884057971|
|                  Mr1| 41.58045977011494|
|              Master3| 5.350833332786958|
|             Officer2|              42.0|
|             Royalty1|              41.6|
|              Master1| 5.306666672229767|
|                 Mrs2| 33.54761904761905|
|              Master2| 2.258888887034522|
|                 Mrs1|              40.4|
|                Miss1| 29.74468085106383|
|                  Mr2| 32.76829268292683|
|             Officer1|50.888888888888886|
+---------------------+------------------+



lastException: Throwable = null


In [28]:
//Age, 根据null age的records对应的Pclass和Name_final分组后的平均来填充缺失age。首先，生成分组key，并获取分组后的平均年龄map。然后广播map,当Age为null时，用udf返回需要填充的值。
val df8 = df7.withColumn("Pclass_Title_key", concat($"Title", $"Pclass"))
val meanAgeMap = df8.groupBy("Pclass_Title_key").mean("Age").map(row => (row.getString(0), row.getDouble(1))).collect().toMap

val meanAgeMap_bc = sc.broadcast(meanAgeMap)
def fillAge: (String => Double) = {
    comb_key => meanAgeMap_bc.value.getOrElse(comb_key, 0.0)
    }
val fillAgeUdf = udf(fillAge)
                           
val df9 = df8.withColumn("Age", when($"Age".isNull, fillAgeUdf($"Pclass_Title_key")).otherwise($"Age"))

//Age_categorized, 对Age进行分类
import org.apache.spark.ml.feature.Bucketizer
val ageBucketBorders = 0.0 +: (10.0 to 60.0 by 5.0).toArray :+ 150.0
val ageBucketer = new Bucketizer().setSplits(ageBucketBorders).setInputCol("Age").setOutputCol("Age_categorized")
val df10 = ageBucketer.transform(df9)

//Age，用mean age填充na
// df.groupBy("Sex", "Pclass", "Name_final"
// val meanAge = df.select(mean($"Age")).first.getDouble(0)
// val df8 = df7.na.fill(meanAge, Seq("Age"))

df8 = [PassengerId: int, Survived: int ... 12 more fields]
meanAgeMap = Map(Officer1 -> 50.888888888888886, Mr2 -> 32.76829268292683, Mr1 -> 41.58045977011494, Mrs1 -> 40.4, Royalty1 -> 41.6, Miss2 -> 22.390625, Mrs2 -> 33.54761904761905, Miss1 -> 29.74468085106383, Master1 -> 5.306666672229767, Officer2 -> 42.0, Miss3 -> 16.1231884057971, Master3 -> 5.350833332786958, Mrs3 -> 33.515151515151516, Master2 -> 2.258888887034522, Mr3 -> 28.662222222222223)
meanAgeMap_bc = Broadcast(10)
fillAge = > Double
fillAgeUdf = UserDefinedFunction(<function1>,DoubleType,Some(List(...


UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))

In [48]:
def transAge(sc: SparkContext, df: Dataset[Row]): Dataset[Row] = {
    val medDF = df.withColumn("Pclass_Title_key", concat($"Title", $"Pclass"))
    val meanAgeMap = medDF.groupBy("Pclass_Title_key").mean("Age").map(row => (row.getString(0), row.getDouble(1))).collect().toMap
    
    val meanAgeMap_bc = sc.broadcast(meanAgeMap)
      
    val fillAgeUdf = udf((comb_key: String) => meanAgeMap_bc.value.getOrElse(comb_key, 0.0))
    
    medDF.withColumn("Age", when($"Age".isNull, fillAgeUdf($"Pclass_Title_key")).otherwise($"Age"))
}

import org.apache.spark.ml.feature.Bucketizer
def categorizeAge(df: Dataset[Row]): Dataset[Row] = {

    val ageBucketBorders = 0.0 +: (10.0 to 60.0 by 5.0).toArray :+ 150.0
    val ageBucketer = new Bucketizer().setSplits(ageBucketBorders).setInputCol("Age").setOutputCol("Age_categorized")
    ageBucketer.transform(df).drop("Pclass_Title_key")   
}

transAge: (sc: org.apache.spark.SparkContext, df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
categorizeAge: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [49]:
val df6 = transAge(sc, df5)

df6 = [PassengerId: int, Survived: int ... 12 more fields]


[PassengerId: int, Survived: int ... 12 more fields]

In [50]:
val df7 = categorizeAge(df6)

df7 = [PassengerId: int, Survived: int ... 12 more fields]


[PassengerId: int, Survived: int ... 12 more fields]

In [30]:
df7.columns.size

13

In [51]:
df7.columns

[PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Title, Age_categorized]

In [29]:
//fellow, 将SibSp和Parch相加，得出同行人数
val df11 = df10.withColumn("fellow", $"SibSp" + $"Parch").drop("SibSp", "Parch")
//size, 对fellow进行分类。此处其实可以留到pipeline部分一次性完成。
val df12 = df11.withColumn("fellow_type", when($"fellow" === 0, "Alone")
                           .when($"fellow" <= 3, "Small")
                           .otherwise("Large"))

df11 = [PassengerId: int, Survived: int ... 12 more fields]
df12 = [PassengerId: int, Survived: int ... 13 more fields]


[PassengerId: int, Survived: int ... 13 more fields]

In [40]:
def createFellow(df: Dataset[Row]): Dataset[Row] = {
    df.withColumn("fellow", $"SibSp" + $"Parch")
}

def categorizeFellow(df: Dataset[Row]): Dataset[Row] = {
    df.withColumn("fellow_type", when($"fellow" === 0, "Alone")
                           .when($"fellow" <= 3, "Small")
                           .otherwise("Large"))
}

lastException: Throwable = null
createFellow: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
categorizeFellow: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [52]:
val df8 = createFellow(df7)
val df9 = categorizeFellow(df8)

df8 = [PassengerId: int, Survived: int ... 13 more fields]
df9 = [PassengerId: int, Survived: int ... 14 more fields]


[PassengerId: int, Survived: int ... 14 more fields]

In [53]:
df9.columns

[PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Title, Age_categorized, fellow, fellow_type]

In [17]:
df.select("Name").filter($"Name".contains("Johnston")).show(false)

+--------------------------------------------+
|Name                                        |
+--------------------------------------------+
|Johnston, Mr. Andrew G                      |
|"Johnston, Miss. Catherine Helen ""Carrie"""|
+--------------------------------------------+



In [30]:
//FName，提取家庭名称。例如："Johnston, Miss. Catherine Helen ""Carrie""" 提取出Johnston，由于spark的读取csv时，如果有引号，读取就会出现多余的引号，所以除了split逗号，还要再split一次引号。
val df13 = df12
  .withColumn("FArray", split($"Name", ","))
  .withColumn("FName", expr("FArray[0]"))
  .withColumn("FArray", split($"FName", "\""))
  .withColumn("FName", $"FArray"(size($"FArray").minus(1)))

//family_type，分为三类，第一类是60岁以下女性遇难的家庭，第二类是18岁以上男性存活的家庭，第三类其他。
val femaleDiedFamily_filter = $"Sex" === "female" and $"Age" < 60 and $"Survived" === 0 and $"fellow" > 0

val maleSurvivedFamily_filter = $"Sex" === "male" and $"Age" >= 18 and $"Survived" === 1 and $"fellow" > 1

val df14 = df13.withColumn("family_type", when(femaleDiedFamily_filter, 1)
                           .when(maleSurvivedFamily_filter, 2).otherwise(0))
//familyTable，家庭分类名单，用于后续test集的转化。此处用${FName}_${family_type}的形式保存。
df14.filter($"family_type".isin(1,2))
  .select(concat($"FName", lit("_"), $"family_type"))
  .dropDuplicates()
  .write.format("text").mode("overwrite").save("familyTable")

//如果需要直接收集成Map的话，可用下面代码。此代码先利用mapPartitions对各分块的数据进行聚合，降低直接调用count而使driver挂掉的风险。另外新建一个默认Set是为了防止某个partition并没有数据的情况（出现概率可能比较少），从而使得Set的类型变为Set[_>:Tuple]而不能直接flatten
// val familyMap = df10
//   .filter($"family_type" === 1 || $"family_type" === 2)
//   .select("FName", "family_type")
//   .rdd
//   .mapPartitions{iter => {
//       if (!iter.isEmpty) {
//       Iterator(iter.map(row => (row.getString(0), row.getInt(1))).toSet)}
//       else Iterator(Set(("defualt", 9)))}
//                 }
//   .collect()
//   .flatten
//   .toMap

// val l1 = df10
//   .filter($"family_type" === 1)
//   .select("FName")
//   .map(name => name.getString(0))
//   .collect.toList

// val l2 = df10
//   .filter($"family_type" === 2)
//   .select("FName")
//   .map(name => name.getString(0))
//   .collect.toList

// val familyMap = l1.map((_, 1)).++(l2.map((_, 2))).toMap

// val familyMap_bc = sc.broadcast(familyMap)
// def familyAdjust: (String => Int) = {
//     family => familyMap_bc.value.getOrElse(family, 0)}
// val familyAdjustUdf = udf(familyAdjust)

// val df11 = df10.withColumn("family_type", familyAdjustUdf($"FName"))
//     .drop("Name")

df13 = [PassengerId: int, Survived: int ... 15 more fields]
femaleDiedFamily_filter = ((((Sex = female) AND (Age < 60)) AND (Survived = 0)) AND (fellow > 0))
maleSurvivedFamily_filter = ((((Sex = male) AND (Age >= 18)) AND (Survived = 1)) AND (fellow > 1))
df14 = [PassengerId: int, Survived: int ... 16 more fields]


[PassengerId: int, Survived: int ... 16 more fields]

In [38]:
def extractFName(df: Dataset[Row]): Dataset[Row] = {
  
    if (!df.columns.contains("Survived") || !df.columns.contains("fellow")){
      throw new IllegalArgumentException("Check if the argument is a training set or if this training set contains column named \"fellow\"")
    }
    
    val medDF = df
      .withColumn("FArray", split($"Name", ","))
      .withColumn("FName", expr("FArray[0]"))
      .withColumn("FArray", split($"FName", "\""))
      .withColumn("FName", $"FArray"(size($"FArray").minus(1)))
    
    //family_type，分为三类，第一类是60岁以下女性遇难的家庭，第二类是18岁以上男性存活的家庭，第三类其他。
    val femaleDiedFamily_filter = $"Sex" === "female" and $"Age" < 60 and $"Survived" === 0 and $"fellow" > 0
    
    val maleSurvivedFamily_filter = $"Sex" === "male" and $"Age" >= 18 and $"Survived" === 1 and $"fellow" > 1
    
    val resDF = medDF.withColumn("family_type", when(femaleDiedFamily_filter, 1)
                               .when(maleSurvivedFamily_filter, 2).otherwise(0))
    //familyTable，家庭分类名单，用于后续test集的转化。此处用${FName}_${family_type}的形式保存。
    resDF.filter($"family_type".isin(1,2))
      .select(concat($"FName", lit("_"), $"family_type"))
      .dropDuplicates()
      .write.format("text").mode("overwrite").save("familyTable") 
      
    resDF
}

extractFName: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [54]:
val df10 = extractFName(df9)

df10 = [PassengerId: int, Survived: int ... 17 more fields]


[PassengerId: int, Survived: int ... 17 more fields]

In [16]:
df14.select("Name", "FName").filter($"FName" === "Johnston").show(5)

+--------------------+--------+
|                Name|   FName|
+--------------------+--------+
|Johnston, Mr. And...|Johnston|
|"Johnston, Miss. ...|Johnston|
+--------------------+--------+



In [31]:
//Fare。首先去掉缺失的（test集合中有一个，如果量多的话，也可以像Age那样通过头衔，年龄等因数来推断）
val df15 = df14.na.drop("any", Seq("Fare"))

//Fare_categorized
import org.apache.spark.ml.feature.QuantileDiscretizer
val fareBucketer = new QuantileDiscretizer().setNumBuckets(4).setInputCol("Fare").setOutputCol("Fare_categorized")

val df16 = fareBucketer.fit(df15).transform(df15)

df15 = [PassengerId: int, Survived: int ... 16 more fields]
fareBucketer = quantileDiscretizer_f99b99ba31a3
df16 = [PassengerId: int, Survived: int ... 17 more fields]


[PassengerId: int, Survived: int ... 17 more fields]

In [55]:
import org.apache.spark.ml.feature.QuantileDiscretizer
def transFare(df: Dataset[Row]): Dataset[Row] = {
    val medDF = df.na.drop("any", Seq("Fare"))
    val fareBucketer = new QuantileDiscretizer().setNumBuckets(4).setInputCol("Fare").setOutputCol("Fare_categorized")
    fareBucketer.fit(medDF).transform(medDF)
}

transFare: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [56]:
val df11 = transFare(df10)

df11 = [PassengerId: int, Survived: int ... 18 more fields]


[PassengerId: int, Survived: int ... 18 more fields]

In [57]:
df11.columns

[PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Title, Age_categorized, fellow, fellow_type, FArray, FName, family_type, Fare_categorized]

In [58]:
df11.columns.size

20

In [60]:
val prePipelineDF = df11.select("Survived", "Pclass", "Sex", "Age_categorized", "fellow_type", "Fare_categorized", "Embarked", "Cabin", "Ticket", "Title", "family_type")

prePipelineDF = [Survived: int, Pclass: int ... 9 more fields]


[Survived: int, Pclass: int ... 9 more fields]

In [34]:
prePipelineDF.columns

[Survived, Pclass, Sex, Age_categorized, fellow_type, Fare_categorized, Embarked, Cabin, Ticket, Title, family_type]

In [35]:
prePipelineDF.first()

[0,3,male,3.0,Small,0.0,S,U,0,Mr,0]

In [45]:
prePipelineDF.show(1)

+--------+------+----+---------------+-----------+----------------+--------+-----+------+-----+-----------+
|Survived|Pclass| Sex|Age_categorized|fellow_type|Fare_categorized|Embarked|Cabin|Ticket|Title|family_type|
+--------+------+----+---------------+-----------+----------------+--------+-----+------+-----+-----------+
|       0|     3|male|            3.0|      Small|             0.0|       S|    U|     0|   Mr|          0|
+--------+------+----+---------------+-----------+----------------+--------+-----+------+-----+-----------+
only showing top 1 row



### Pipeline

构建pipeline，包括转换数据的格式，如double，onehot等，划分label和feature，

In [53]:

// +--------+------+----+---------------+-----------+----------------+--------+-----+------+-----+-----------+
// |Survived|Pclass| Sex|Age_categorized|fellow_type|Fare_categorized|Embarked|Cabin|Ticket|Title|family_type|
// +--------+------+----+---------------+-----------+----------------+--------+-----+------+-----+-----------+
// |       0|     3|male|            3.0|      Small|             0.0|       S|    U|     0|   Mr|          0|
// +--------+------+----+---------------+-----------+----------------+--------+-----+------+-----+-----------+

prePipelineDF.cache()

//StringIndexer and OneHotEncoder pipeline
import org.apache.spark.ml.feature.StringIndexer
val stringCols = Array("Sex","fellow_type", "Embarked", "Cabin", "Ticket", "Title")
val subOneHotCols = stringCols.map(cname => s"${cname}_index")
val index_transformers: Array[org.apache.spark.ml.PipelineStage] = stringCols.map(
  cname => new StringIndexer()
    .setInputCol(cname)
    .setOutputCol(s"${cname}_index")
    .setHandleInvalid("skip")
)

import org.apache.spark.ml.feature.OneHotEncoder
val oneHotCols = subOneHotCols ++ Array("Pclass", "Age_categorized", "Fare_categorized", "family_type")
val vectorCols = oneHotCols.map(cname => s"${cname}_encoded")
val encode_transformers: Array[org.apache.spark.ml.PipelineStage] = oneHotCols.map(
  cname => new OneHotEncoder()
    .setInputCol(cname)
    .setOutputCol(s"${cname}_encoded")
)

val pipelineStage = index_transformers ++ encode_transformers
import org.apache.spark.ml.Pipeline
val index_onehot_pipeline = new Pipeline().setStages(pipelineStage)
val index_onehot_pipelineModel = index_onehot_pipeline.fit(prePipelineDF)
val df_indexed = pipelineModel.transform(prePipelineDF)

df_indexed.cache()

//separate and model pipeline，包含划分label和features，机器学习模型的pipeline
import org.apache.spark.ml.feature.VectorAssembler
val vectorAssembler = new VectorAssembler()
  .setInputCols(vectorCols)
  .setOutputCol("features")

import org.apache.spark.ml.classification.RandomForestClassifier
val rfc = new RandomForestClassifier()
    .setLabelCol("Survived")
    .setFeaturesCol("features")
    .setPredictionCol("prediction")

import org.apache.spark.ml.classification.GBTClassifier
val gbtc = new GBTClassifier()
    .setLabelCol("Survived")
    .setFeaturesCol("features")
    .setPredictionCol("prediction")

import org.apache.spark.ml.Pipeline
val pipeline = new Pipeline().setStages(Array(vectorAssembler, gbtc))

stringCols = Array(Sex, fellow_type, Embarked, Cabin, Ticket, Title)
subOneHotCols = Array(Sex_index, fellow_type_index, Embarked_index, Cabin_index, Ticket_index, Title_index)
index_transformers = Array(strIdx_e325b470f2a7, strIdx_1afd4e18f157, strIdx_f35f19bed306, strIdx_c65c155911a2, strIdx_3c51a6aaf946, strIdx_e1a2e89d57d1)
oneHotCols = Array(Sex_index, fellow_type_index, Embarked_index, Cabin_index, Ticket_index, Title_index, Pclass, Age_categorized, Fare_categorized, family_type)
vectorCols = Array(Sex_index_encoded, fellow_type_index_encoded, Embarked_index_encoded, Cabin_index_encod...


[Sex_index_encoded, fellow_type_index_encoded, Embarked_index_encoded, Cabin_index_encoded, Ticket_index_encoded, Title_index_encoded, Pclass_encoded, Age_categorized_encoded, Fare_categorized_encoded, family_type_encoded]

In [69]:
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoder
import org.apache.spark.ml.Pipeline

def index_onehot(df: Dataset[Row]): Tuple2[Dataset[Row], Array[String]] = {
    val stringCols = Array("Sex","fellow_type", "Embarked", "Cabin", "Ticket", "Title")
    val subOneHotCols = stringCols.map(cname => s"${cname}_index")
    val index_transformers: Array[org.apache.spark.ml.PipelineStage] = stringCols.map(
      cname => new StringIndexer()
        .setInputCol(cname)
        .setOutputCol(s"${cname}_index")
        .setHandleInvalid("skip")
    )
    
    val oneHotCols = subOneHotCols ++ Array("Pclass", "Age_categorized", "Fare_categorized", "family_type")
    val vectorCols = oneHotCols.map(cname => s"${cname}_encoded")
    val encode_transformers: Array[org.apache.spark.ml.PipelineStage] = oneHotCols.map(
      cname => new OneHotEncoder()
        .setInputCol(cname)
        .setOutputCol(s"${cname}_encoded")
    )
    
    val pipelineStage = index_transformers ++ encode_transformers
    val index_onehot_pipeline = new Pipeline().setStages(pipelineStage)
    val index_onehot_pipelineModel = index_onehot_pipeline.fit(df)
    
    val resDF = index_onehot_pipelineModel.transform(df).drop(stringCols:_*).drop(subOneHotCols:_*)
    (resDF, vectorCols)
}

index_onehot: (df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])(org.apache.spark.sql.Dataset[org.apache.spark.sql.Row], Array[String])


In [70]:
val (df_indexed, cols) = index_onehot(prePipelineDF)
df_indexed.cache()

df_indexed = [Survived: int, Pclass: int ... 13 more fields]
cols = Array(Sex_index_encoded, fellow_type_index_encoded, Embarked_index_encoded, Cabin_index_encoded, Ticket_index_encoded, Title_index_encoded, Pclass_encoded, Age_categorized_encoded, Fare_categorized_encoded, family_type_encoded)


[Survived: int, Pclass: int ... 13 more fields]

In [73]:
df_indexed.columns

[Survived, Pclass, Age_categorized, Fare_categorized, family_type, Sex_index_encoded, fellow_type_index_encoded, Embarked_index_encoded, Cabin_index_encoded, Ticket_index_encoded, Title_index_encoded, Pclass_encoded, Age_categorized_encoded, Fare_categorized_encoded, family_type_encoded]

In [72]:
df_indexed.columns.size

15

In [39]:
df_indexed.select("Sex_index_encoded", "fellow_type_index_encoded", "Embarked_index_encoded", "Cabin_index_encoded", "Ticket_index_encoded", "Title_index_encoded", "Pclass_encoded", "Age_categorized_encoded", "Fare_categorized_encoded", "family_type_encoded").take(5)

"(1,[0],[1.0])","(2,[1],[1.0])","(2,[0],[1.0])","(8,[0],[1.0])","(39,[0],[1.0])","(5,[0],[1.0])","(3,[],[])","(11,[3],[1.0])","(3,[0],[1.0])","(2,[0],[1.0])"
"(1,[],[])","(2,[1],[1.0])","(2,[1],[1.0])","(8,[1],[1.0])","(39,[7],[1.0])","(5,[2],[1.0])","(3,[1],[1.0])","(11,[6],[1.0])","(3,[],[])","(2,[0],[1.0])"
"(1,[],[])","(2,[0],[1.0])","(2,[0],[1.0])","(8,[0],[1.0])","(39,[3],[1.0])","(5,[1],[1.0])","(3,[],[])","(11,[4],[1.0])","(3,[1],[1.0])","(2,[0],[1.0])"
"(1,[],[])","(2,[1],[1.0])","(2,[0],[1.0])","(8,[1],[1.0])","(39,[4],[1.0])","(5,[2],[1.0])","(3,[1],[1.0])","(11,[6],[1.0])","(3,[],[])","(2,[0],[1.0])"
"(1,[0],[1.0])","(2,[0],[1.0])","(2,[0],[1.0])","(8,[0],[1.0])","(39,[0],[1.0])","(5,[0],[1.0])","(3,[],[])","(11,[6],[1.0])","(3,[1],[1.0])","(2,[0],[1.0])"


In [47]:
df_indexed.select("prediction","probability","rawPrediction").show

+----------+--------------------+--------------------+
|prediction|         probability|       rawPrediction|
+----------+--------------------+--------------------+
|       0.0|[0.81312048921093...|[16.2624097842186...|
|       1.0|[0.06949345575278...|[1.38986911505578...|
|       1.0|[0.42792627436304...|[8.55852548726090...|
|       1.0|[0.11751845835005...|[2.35036916700102...|
|       0.0|[0.85582635592717...|[17.1165271185434...|
|       0.0|[0.83015715429849...|[16.6031430859699...|
|       1.0|[0.46082803361221...|[9.21656067224423...|
|       0.0|[0.70185081605966...|[14.0370163211933...|
|       1.0|[0.23987472789878...|[4.79749455797566...|
|       1.0|[0.16763934116505...|[3.35278682330107...|
|       1.0|[0.27560241875773...|[5.51204837515475...|
|       1.0|[0.29037471227365...|[5.80749424547318...|
|       0.0|[0.83574358951282...|[16.7148717902565...|
|       0.0|[0.83795243577368...|[16.7590487154736...|
|       1.0|[0.45964589961478...|[9.19291799229568...|
|       1.

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder
// val paramGrid = new ParamGridBuilder()
//   .addGrid(rfc.numTrees, Seq(50, 100, 150, 200))
//   .addGrid(rfc.maxDepth, Seq(4, 5, 7, 10))
//   .build()

val paramGrid = new ParamGridBuilder()
  .addGrid(gbtc.stepSize, Seq(0.03, 0.07, 0.13, 0.17))
  .addGrid(gbtc.maxDepth, Seq(4, 5, 7, 10))
  .addGrid(gbtc.maxIter, Seq(150, 200))
  .build()

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator = new BinaryClassificationEvaluator()
  .setMetricName("areaUnderROC")
  .setRawPredictionCol("prediction")
  .setLabelCol("Survived")

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val multiclassEval = new MulticlassClassificationEvaluator()
  .setLabelCol("Survived")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

import org.apache.spark.ml.tuning.TrainValidationSplit
val tvs = new TrainValidationSplit()
  .setTrainRatio(0.9) 
  .setEstimatorParamMaps(paramGrid)
  .setEstimator(pipeline)
  .setEvaluator(multiclassEval)

//    val pipeline = new Pipeline().setStages(Array(assembler, indexer, classifier))

//     val paramGrid = new ParamGridBuilder().
//       addGrid(classifier.minInfoGain, Seq(0.0, 0.05)).
//       addGrid(classifier.numTrees, Seq(1, 10)).
//       build()

//     val multiclassEval = new MulticlassClassificationEvaluator().
//       setLabelCol("Cover_Type").
//       setPredictionCol("prediction").
//       setMetricName("accuracy")

//     val validator = new TrainValidationSplit().
//       setSeed(Random.nextLong()).
//       setEstimator(pipeline).
//       setEvaluator(multiclassEval).
//       setEstimatorParamMaps(paramGrid).
//       setTrainRatio(0.9)

In [42]:
val validatorModel = tvs.fit(df_indexed)

validatorModel = tvs_a07660574bcd


tvs_a07660574bcd

In [ ]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{TrainValidationSplit, TrainValidationSplitModel}

def trainData(df: Dataset[Row], vectorCols: Array[String]): TrainValidationSplitModel = {
    //separate and model pipeline，包含划分label和features，机器学习模型的pipeline
    val vectorAssembler = new VectorAssembler()
      .setInputCols(vectorCols)
      .setOutputCol("features")
    
    val gbtc = new GBTClassifier()
        .setLabelCol("Survived")
        .setFeaturesCol("features")
        .setPredictionCol("prediction")
    
    val pipeline = new Pipeline().setStages(Array(vectorAssembler, gbtc))
    
    val paramGrid = new ParamGridBuilder()
    .addGrid(gbtc.stepSize, Seq(0.03, 0.07, 0.13))
    .addGrid(gbtc.maxDepth, Seq(5, 7, 10))
    .addGrid(gbtc.maxIter, Seq(150, 200))
    .build()
    
    val multiclassEval = new MulticlassClassificationEvaluator()
      .setLabelCol("Survived")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")
    
    val tvs = new TrainValidationSplit()
      .setTrainRatio(0.9) 
      .setEstimatorParamMaps(paramGrid)
      .setEstimator(pipeline)
      .setEvaluator(multiclassEval)
    
    tvs.fit(df)
}

In [102]:
val validatorModel = trainData(df_indexed, cols)

validatorModel = tvs_e366fa3a7918


tvs_e366fa3a7918

In [43]:
import org.apache.spark.ml.PipelineModel
val bestModel = validatorModel.bestModel
println(bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

{
	rfc_2715e7422000-cacheNodeIds: false,
	rfc_2715e7422000-checkpointInterval: 10,
	rfc_2715e7422000-featureSubsetStrategy: auto,
	rfc_2715e7422000-featuresCol: features,
	rfc_2715e7422000-impurity: gini,
	rfc_2715e7422000-labelCol: Survived,
	rfc_2715e7422000-maxBins: 32,
	rfc_2715e7422000-maxDepth: 7,
	rfc_2715e7422000-maxMemoryInMB: 256,
	rfc_2715e7422000-minInfoGain: 0.0,
	rfc_2715e7422000-minInstancesPerNode: 1,
	rfc_2715e7422000-numTrees: 100,
	rfc_2715e7422000-predictionCol: prediction,
	rfc_2715e7422000-probabilityCol: probability,
	rfc_2715e7422000-rawPredictionCol: rawPrediction,
	rfc_2715e7422000-seed: 207336481,
	rfc_2715e7422000-subsamplingRate: 1.0
}


bestModel = pipeline_e7d6af4e8350


pipeline_e7d6af4e8350

In [ ]:
val paramsAndMetrics = validatorModel.validationMetrics
  .zip(validatorModel.getEstimatorParamMaps)
  .sortBy(-_._1)

paramsAndMetrics.foreach { case (metric, params) => 
    println(metric)
    println(params)
    println() 
}

In [ ]:
bestModel.write.overwrite().save("Titanic_gbtc")